# Notebook for Training Sorting Model on Colab

**Note:**
- This notebook is here for documentation reasons
- It was executed on Google Colab
- The training was based on SVI images manually sorted into usable or unusable for conducting a survey on rating SVI based on bicycle-friendliness

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
import os
from PIL import Image
import shutil
import zipfile

import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score
from tqdm import tqdm
from copy import deepcopy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define a simple dataset class
class StreetViewImagesDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []  # 0 for manually_deleted, 1 for manually_kept
        for label, subdir in enumerate(["manually_deleted", "manually_kept"]):
            subdir_path = os.path.join(root_dir, subdir)
            for img_name in os.listdir(subdir_path):
                self.images.append(os.path.join(subdir_path, img_name))
                self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path)
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Define your transforms
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
os.chdir('/content/drive/My Drive/2024_Data_Science_Project')

# `train_sorting` is the root directory containing 'manually_kept' and 'manually_deleted'
dataset = StreetViewImagesDataset(root_dir='train_sorting', transform=transform)

# Splitting dataset into train and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
# Use a pre-trained model and modify it for binary classification
model = models.mobilenet_v2(pretrained=True)
# Replace the classifier layer
model.classifier[1] = nn.Linear(model.last_channel, 2)  # 2 classes: manually_kept and manually_deleted

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.0008)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Define Training

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=25, patience=4):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    best_model_wts = deepcopy(model.state_dict())
    best_loss = np.inf
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                data_loader = train_loader
            else:
                model.eval()   # Set model to evaluate mode
                data_loader = val_loader

            running_loss = 0.0
            running_corrects = 0
            all_preds = []
            all_labels = []

            # Add a tqdm progress bar
            progress_bar = tqdm(data_loader, desc=f"{phase.capitalize()} Phase", leave=False)

            # Iterate over data
            for inputs, labels in progress_bar:
                inputs, labels = inputs.to(device), labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

                # Update the progress bar description with the current loss
                progress_bar.set_description(f"{phase.capitalize()} Phase - Loss: {loss.item():.4f}")

            epoch_loss = running_loss / len(data_loader.dataset)
            epoch_acc = running_corrects.double() / len(data_loader.dataset)
            epoch_prec = precision_score(all_labels, all_preds)
            epoch_recall = recall_score(all_labels, all_preds)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} Precision: {epoch_prec:.4f} Recall: {epoch_recall:.4f}')

            # Deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = deepcopy(model.state_dict())
                epochs_no_improve = 0
            elif phase == 'val':
                epochs_no_improve += 1

        if epochs_no_improve == patience:
            print("Early stopping")
            break

        print()

    print('Best val loss: {:4f}'.format(best_loss))

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

### Train

In [ ]:
best_mobilenetv2_model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=25, patience=10)

Epoch 1/25
----------


train Loss: 0.6231 Acc: 0.8243 Precision: 0.8684 Recall: 0.9267


val Loss: 0.2201 Acc: 0.9000 Precision: 0.9128 Recall: 0.9691

Epoch 2/25
----------


train Loss: 0.1988 Acc: 0.9159 Precision: 0.9482 Recall: 0.9496


val Loss: 0.1657 Acc: 0.9300 Precision: 0.9625 Recall: 0.9506

Epoch 3/25
----------


...

val Loss: 0.2190 Acc: 0.9200 Precision: 0.9506 Recall: 0.9506

Epoch 11/25
----------


train Loss: 0.0798 Acc: 0.9737 Precision: 0.9847 Recall: 0.9832


val Loss: 0.2682 Acc: 0.9000 Precision: 0.9383 Recall: 0.9383

Epoch 12/25
----------


train Loss: 0.0508 Acc: 0.9824 Precision: 0.9908 Recall: 0.9878


val Loss: 0.2977 Acc: 0.9150 Precision: 0.9341 Recall: 0.9630
Early stopping
Best val loss: 0.165657


### Save the Model

In [ ]:
models_dir = 'models'
model_path = os.path.join(models_dir, 'img_sorting_mobilenetV2.pth')
torch.save(best_mobilenetv2_model.state_dict(), model_path)